In [ ]:
from bbg_data.core.session import session
from bbg_data.api.options import OptionChainFetcher
from typing import Any
import pandas as pd
import datetime as dt
from bbg_data.api import fetch_atm_option_timeseries

In [ ]:
AS_OF_DATE = dt.date(2025,1,27)

In [ ]:
with session() as bbg_session:
    fetcher = OptionChainFetcher(bbg_session)
    nvidia_option_tickers = fetcher.get_option_chain("NVDA US Equity", as_of_date=AS_OF_DATE)
    print(f"Found {len(nvidia_option_tickers)} option tickers for NVDA US Equity")
    nvidia_options_all_data = fetcher.get_option_market_data(
        tickers=nvidia_option_tickers, as_of_date=AS_OF_DATE
    )

In [ ]:
nvidia_options_all_data

In [ ]:
records: list[dict[str, Any]] = []
for data in nvidia_options_all_data:
    record = {
        "security": data.contract.ticker,
        "strike": data.contract.strike,
        "expiry": data.contract.expiry,
        "option_type": data.contract.option_type.value,
        "settlement_price": data.settlement_price,
        "last_price": data.last_price,
        "bid": data.bid,
        "ask": data.ask,
        "mid_price": data.mid_price,
        "volume": data.volume,
        "open_interest": data.open_interest,
        "implied_vol": data.implied_vol,
        "delta": data.delta,
        "gamma": data.gamma,
        "theta": data.theta,
        "vega": data.vega,
    }
    records.append(record)

nvidia_options_df = pd.DataFrame(records)

In [ ]:
nvidia_options_df.to_csv(f"nvda_options_data_{AS_OF_DATE.isoformat()}.csv", index=False)

In [ ]:
nvidia_atm_df = fetch_atm_option_timeseries(
    underlying="NVDA US Equity",
    start_date=dt.date(2025, 1, 1),
    end_date=dt.date(2025, 3, 31),
)

In [ ]:
nvidia_atm_df.to_csv("nvda_atm.csv", index=False)